In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
from datetime import datetime
from fedai.servers import *
from fedai.clients import *
from fedai.utils import *
from fedai.trainers import LLMTrainer
from omegaconf import DictConfig, OmegaConf
import hydra
import argparse


#### Hydra as our config manager

In [ ]:
path = 'folder/config/config.yaml'
config_path = os.path.dirname(path)
config_name = os.path.basename(path)
config_name = os.path.splitext(config_name)[0]

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')

fname = parser.parse_args().fname
config_path = os.path.dirname(fname)
config_name = os.path.splitext(os.path.basename(config_path))[0]

In [ ]:
@hydra.main(version_base=None, config_path=config_path, config_name=config_name)
def app(cfg: DictConfig) -> None:
    print(OmegaConf.to_yaml(cfg))
    loss_ds, gener_ds = load_ds(cfg)
    list_train_ds, list_eval_ds, tokenizer, datacollator = loss_ds
    list_train_ds_genr, list_eval_ds_genr = gener_ds
    

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/ahmed/.local/share/jupyter/runtime/kernel-v345b718cf6313858386357bb42dd52bcafd1180c9.json


SystemExit: 2

/home/ahmed/miniconda3/envs/fedai/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Test main


In [ ]:
from fedai.core import *

In [ ]:
# from fedai.data import get_dolly
# from transformers import AutoTokenizer
# import wandb
# import pandas as pd
# tokenizer = AutoTokenizer.from_pretrained('gpt2')
# tokenizer.pad_token = tokenizer.eos_token
# dolly = get_dolly('', tokenizer)

In [ ]:
# (lst_train_ds, lst_eval_set, tokenizer, data_collator), (lst_train_ds_genr, lst_eval_set_genr)  = dolly

In [ ]:
# lst_data_dict = [{'train': tr, 'test': test} for tr, test in zip(lst_train_ds, lst_eval_set)]
# lst_data_dict_genr = [{'train': tr, 'test': test} for tr, test in zip(lst_train_ds_genr, lst_eval_set_genr)]

In [ ]:
# lst_data_dict[0]

{'train': <torch.utils.data.dataset.Subset>,
 'test': <torch.utils.data.dataset.Subset>}

In [ ]:
# lst_data_dict[0]['train'].dataset[0].keys()

dict_keys(['input_ids', 'labels', 'categories', 'tasks'])

In [ ]:
# len(lst_data_dict) == len(lst_data_dict_genr)

True

In [ ]:
# def criterion(y_pred, y_true= None):
#     loss = y_pred.loss
#     return loss

In [ ]:
# kwargs = {'lst_gen_data_dict': lst_data_dict_genr,
#           'tokenizer': tokenizer,
#           'collat_fn': data_collator,
#           'criterion': criterion
#           }

In [ ]:
from fedai.core import get_cfg
cfg = get_cfg()
cfg

{'dataset': 'dolly', 'model': 'openai-community/gpt2', 'max_length': 1022, 'num_clients': 80, 'iid': 'dir0.5', 'batch_size': 1, 'dataset_subsample': 1.0, 'eval_metric': 'loss', 'log_root': 'logs', 'save_dir': 'checkpoints', 'lora_alpha': 8, 'lora_dropout': 0.05, 'r': 32, 'target_modules': ['c_attn'], 'lr': 5e-05, 'optimizer': 'Adam', 'name': 'mira', 'device': 0, 'bias_sampling': False, 'num_clients_per_task': 10, 'use_prompts': True, 'rounds': 40, 'm': 0.05}

In [ ]:
cfg.log_dir = os.path.join(cfg.log_root, 
                            datetime.now().strftime("%Y%m%d_%H%M%S"))
        
cfg.save_dir = os.path.join(cfg.save_dir, 
                            datetime.now().strftime("%Y%m%d_%H%M%S"))

In [ ]:
cfg.training_metrics = ['accuracy']
cfg.test_metrics = ["rouge"]

In [ ]:
cfg.max_new_tokens = 128
cfg.grad_norm_clip = 5.0
cfg.local_epochs = 1


In [ ]:
# len(lst_data_dict)# number of clients

80

In [ ]:
# server = get_server(cfg, lst_data_dict, model= None, holdout_ds= None, **kwargs)  # noqa: F405

/home/ahmed/miniconda3/envs/fedai/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# server.client_list

<fedai.utils.LazyList>

In [ ]:
# client_indices_rounds = get_client_indices_rounds(cfg)

In [ ]:
# len(client_indices_rounds)

40

In [ ]:
# run = wandb.init(project=cfg.project, name= cfg.exp_name, config=cfg)

In [ ]:
# for t in range(1, 41):
#     clients_history = []
#     local_ds_len = {}
#     prev_clients = set()    
#     for client in server.get_selected_client(client_indices_rounds[t-1]):
#         # get the latest model from the server.
#         client.model = server.send(client)
#         # intialize the local training
#         client.init_local_train("saved_dir")
#         # train the model
#         trainer = LLMTrainer(client)
#         history = trainer.train()
#         # save the model to the server
#         client.model, local_ds_len, prev_clients, _ = client.terminate_local_train(t, local_ds_len, prev_clients)
#         clients_history.append(history)
#         # Save space and delete the client (its state is already saved)
#         print(f"client {client.idx} finished round {t}")
#         server.save_space(client)

#     round_metrics = wandb.Table(dataframe= pd.DataFrame(clients_history))
#     # averages the client metrics history
#     clients_avg_metrics= {k: sum([h[k] for h in clients_history]) / len(clients_history) for k in clients_history[0].keys()}
#     # log all average metrics and the table of local clients metrics
#     run.log(clients_avg_metrics.update({"round {t} Local Metrics": round_metrics}))

****************************************
Inside the test () function of client 47
Client 47's Batch loss inside eval() : 2.7816171646118164
Client 47's Batch loss inside eval() : 6.8650221824646
Client 47's Batch loss inside eval() : 3.182016134262085
Client 47's Batch loss inside eval() : 6.86127233505249
Client 47's Batch loss inside eval() : 3.5523483753204346
Client 47's Batch loss inside eval() : 6.1157755851745605
Client 47's Batch loss inside eval() : 3.8836827278137207
Client 47's Batch loss inside eval() : 4.441784858703613
Client 47's Batch loss inside eval() : 2.132840633392334
Client 47's Batch loss inside eval() : 4.154135227203369
Client 47's Batch loss inside eval() : 3.746839761734009
Client 47's Batch loss inside eval() : 3.0870888233184814
Client 47's Batch loss inside eval() : 2.899362325668335
Client 47's Batch loss inside eval() : 3.435415267944336
Client 47's Batch loss inside eval() : 2.828298568725586
Client 47's Batch loss inside eval() : 2.951650857925415
Clie

Batch loss is 3.3085391521453857


Batch loss is 3.165648937225342


Batch loss is 3.0138185024261475


Batch loss is 3.844353437423706


...

Batch loss is 2.110776901245117


Batch loss is 3.1941959857940674


Batch loss is 3.741204023361206


Batch loss is 4.00759744644165


: 